### **Gaussian Splatting for Hayden Library Lantern**

In [ ]:
!git clone https://github.com/ekagra1602/GaussianSplatting
%cd GaussianSplatting

In [ ]:
!pip install -r requirements.txt
!apt-get -qq update && apt-get -qq install -y ffmpeg

**Prep Dataset**

In [ ]:
!python scripts/dataset_prep.py \
  --video /content/ASU_Lantern_Orbit.mp4 \
  --out lantern_ds \
  --target_frames 150 \
  --min_sharpness 80 \
  --width 1600

**Run VGGT to predict camera poses, depth, and geometry**

In [ ]:
!git clone https://github.com/facebookresearch/vggt.git /content/vggt

In [ ]:
%cd /content/vggt

In [ ]:
!python -c "import vggt; print('VGGT installed!')"

In [ ]:
pip install hydra-core omegaconf

In [ ]:
!mkdir -p /content/GaussianSplatting/lantern_ds/images_full_backup
!cp /content/GaussianSplatting/lantern_ds/images/*.jpg /content/GaussianSplatting/lantern_ds/images_full_backup/
!echo "Backup complete. Original images saved."

In [ ]:
import os
import glob

# Get all images sorted
image_dir = "/content/GaussianSplatting/lantern_ds/images"
images = sorted(glob.glob(f"{image_dir}/*.jpg"))

print(f"Total images: {len(images)}")

# Keep every 2nd image, delete the rest
for i, img_path in enumerate(images):
    if i % 2 == 1:  # Delete odd indices (keep even: 0, 2, 4, 6...)
        os.remove(img_path)

# Check remaining count
remaining = sorted(glob.glob(f"{image_dir}/*.jpg"))
print(f"Remaining images: {len(remaining)}")

In [ ]:
!pip uninstall pycolmap -y
!pip install pycolmap==3.10.0

In [ ]:
!python demo_colmap.py --scene_dir=/content/GaussianSplatting/lantern_ds

In [ ]:
!ls -lh /content/GaussianSplatting/lantern_ds/sparse/

## **Restore and Backup Session**

In [ ]:
# restore_session.py - Start of every new session
import os
from google.colab import drive

print("🔄 Restoring session...")

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Smart repo handling
REPO_URL = "https://github.com/ekagra1602/GaussianSplatting"
REPO_DIR = "/content/GaussianSplatting"

if not os.path.exists(REPO_DIR):
    print("📥 Cloning repository (first time)...")
    !git clone {REPO_URL}
else:
    print("✅ Repository exists, pulling latest changes...")
    %cd {REPO_DIR}
    !git pull origin main || echo "⚠️  Pull failed (maybe no internet or no changes)"

%cd {REPO_DIR}

# 3. Restore dataset from Google Drive backup
print("📦 Restoring lantern_ds dataset...")
if os.path.exists('/content/drive/MyDrive/GaussianSplatting_Backup/lantern_ds'):
    !cp -r /content/drive/MyDrive/GaussianSplatting_Backup/lantern_ds {REPO_DIR}/
    print("✅ Dataset restored from Drive")
else:
    print("❌ Backup not found in Drive! Check your backup location.")

# 4. Reinstall dependencies
print("📚 Installing dependencies...")
!pip install -q hydra-core omegaconf pycolmap==3.10.0

# 5. Comprehensive verification
print("\n🔍 Verification:")
checks = {
    "Repository": os.path.exists(REPO_DIR),
    "lantern_ds": os.path.exists(f"{REPO_DIR}/lantern_ds"),
    "images": os.path.exists(f"{REPO_DIR}/lantern_ds/images"),
    "sparse": os.path.exists(f"{REPO_DIR}/lantern_ds/sparse"),
    "COLMAP files": os.path.exists(f"{REPO_DIR}/lantern_ds/sparse/cameras.bin")
}

for item, exists in checks.items():
    status = "✅" if exists else "❌"
    print(f"  {status} {item}")

if checks["sparse"]:
    sparse_files = os.listdir(f"{REPO_DIR}/lantern_ds/sparse/")
    print(f"\n📁 Sparse folder contains: {sparse_files}")

print(f"\n📂 Working directory: {os.getcwd()}")
print("\n🚀 Session fully restored! Ready to continue.")

In [ ]:
# 💾 BACKUP CELL
from google.colab import drive
import os

# Mount Drive (safe to run multiple times)
drive.mount('/content/drive')

WORK_DIR = "/content/GaussianSplatting/lantern_ds"
BACKUP_DIR = "/content/drive/MyDrive/GaussianSplatting_Backup"

print("💾 Starting backup...")

# Create backup directory
!mkdir -p {BACKUP_DIR}

# Sync with progress
!rsync -av --progress {WORK_DIR}/ {BACKUP_DIR}/lantern_ds/

# Show what was backed up
print("\n📊 Backup Summary:")
!du -sh {BACKUP_DIR}/lantern_ds/
print(f"📁 Backed up to: {BACKUP_DIR}/lantern_ds/")

# List major folders
print("\n📂 Backed up folders:")
for folder in ['images', 'sparse', 'outputs', 'checkpoints']:
    folder_path = f"{BACKUP_DIR}/lantern_ds/{folder}"
    if os.path.exists(folder_path):
        size = !du -sh {folder_path} | cut -f1
        print(f"  ✅ {folder}: {size[0]}")
    else:
        print(f"  ⚪ {folder}: (not yet created)")

print("\n✅ Backup complete! Safe to disconnect.")

# **Run Gaussian Spplatting on output from VGGT**

In [ ]:
# Install gsplat (recommended version from VGGT docs)
!pip install gsplat==1.3.0

# Install additional dependencies
!pip install \
  opencv-python \
  pillow \
  tqdm \
  lpips \
  trimesh \
  plyfile

print("✅ gsplat installed!")

In [ ]:
# Clone gsplat repo to get example training scripts
%cd /content
!git clone https://github.com/nerfstudio-project/gsplat.git
%cd gsplat

# Install example requirements
!pip install -r examples/requirements.txt

print("✅ gsplat repo cloned!")

In [ ]:
import os
import shutil

# Create the 0 subdirectory
sparse_dir = "/content/GaussianSplatting/lantern_ds/sparse"
sparse_0_dir = f"{sparse_dir}/0"

os.makedirs(sparse_0_dir, exist_ok=True)

# Move all .bin and .ply files into 0/
for file in os.listdir(sparse_dir):
    if file.endswith(('.bin', '.ply')):
        src = f"{sparse_dir}/{file}"
        dst = f"{sparse_0_dir}/{file}"
        shutil.move(src, dst)
        print(f"Moved: {file} → sparse/0/{file}")

print("\n✅ Dataset structure fixed!")

In [ ]:
!echo "=== Checking dataset structure ==="
!ls -lh /content/GaussianSplatting/lantern_ds/
!echo ""
!ls -lh /content/GaussianSplatting/lantern_ds/sparse/0/
!echo ""
!ls /content/GaussianSplatting/lantern_ds/images/ | head -5

In [ ]:
# Create the missing __init__.py
!touch /content/gsplat/examples/datasets/__init__.py

print("Created __init__.py in datasets/")

In [ ]:
# Uninstall old version
!pip uninstall -y gsplat

# Install latest version
!pip install gsplat

print("✅ gsplat upgraded to latest version")

In [ ]:
%cd /content/gsplat/examples

!python simple_trainer.py default \
  --data_dir /content/GaussianSplatting/lantern_ds \
  --result_dir /content/GaussianSplatting/results/lantern_baseline \
  --data_factor 1 \
  --max_steps 10000 \
  --save_ply

In [ ]:
!echo "=== PLY Files ==="
!ls -lh /content/GaussianSplatting/results/lantern_baseline/ply/

!echo ""
!echo "=== Rendered Videos ==="
!ls -lh /content/GaussianSplatting/results/lantern_baseline/videos/

In [ ]:
#Download PLY File
from google.colab import files
files.download("/content/GaussianSplatting/results/lantern_30k_v2/ply/point_cloud_29999.ply")

In [ ]:
%cd /content/gsplat

!python examples/simple_trainer.py default \
  --data-dir /content/GaussianSplatting/lantern_ds \
  --result-dir /content/GaussianSplatting/results/lantern_30k_v2 \
  --data-factor 1 \
  --max-steps 30000 \
  --save-ply \
  --opacity-reg 0.02 \
  --scale-reg 0.02 \
  --random-bkgd \
  --disable-viewer \
  --strategy.prune-opa 0.01 \
  --strategy.prune-scale3d 0.05

In [ ]:
%cd /content/gsplat

# See parameters for the 'default' config
!python examples/simple_trainer.py default --help